## Prediction Problem Code Template

- Please only put down the best setup, which means the preprocessing, predictor selection (if applicable), tuning and training, and prediction. The best setup means the best performance on Kaggle Public Leaderboard.
- **Keep in mind that your code needs to run for credit.**
- Choose the problem type and the model type below, before your code. You will use this template for all parts of both regression and classification problems.

**Problem:** Classification

**Model:** Boosting

**Group Members:** Ben Maisel

**Performance on Kaggle:** 0.92

### Libraries

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict, RepeatedKFold
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier, \
RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import itertools as it
import time as time
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

import warnings
from IPython import display
import re

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostRegressor, CatBoostClassifier

### Data and Preprocessing

In [7]:
train = pd.read_csv('train_classification.csv')
test = pd.read_csv('test_classification.csv')
train.head()
# train.columns
h_id = test.host_id
test.head()

id     host_id  host_since host_location  \
0  1543972437713169913  1521551465  2020-12-16           NaN   
1  1710552057351883447   429737865  2016-12-16  New York, NY   
2             97075525   995041819  2019-03-14           NaN   
3             83734823    97552339  2014-12-02   Chicago, IL   
4             56722823  1029857633  2019-04-22           NaN   

  host_response_time host_response_rate host_acceptance_rate  \
0       within a day                88%                  80%   
1     within an hour               100%                  97%   
2     within an hour                99%                  99%   
3     within an hour               100%                 100%   
4       within a day               100%                  65%   

    host_neighbourhood  host_listings_count  host_total_listings_count  ...  \
0       Near West Side                   44                         44  ...   
1            Cambridge                 4820                       6438  ...   
2  West Loop/Greektown                   71                         88  ...   
3        Streeterville                   14                         47  ...   
4          River North                   20                         22  ...   

  review_scores_checkin review_scores_communication review_scores_location  \
0                   NaN                         NaN                    NaN   
1                   NaN                         NaN                    NaN   
2                  4.82                        4.91                   4.91   
3                  4.67                        4.83                   5.00   
4                  4.67                        4.67                   5.00   

  review_scores_value  instant_bookable  calculated_host_listings_count  \
0                 NaN                 f                              44   
1                 NaN                 f                             643   
2                4.64                 t                              51   
3                4.67                 f                              14   
4                4.67                 f                              19   

  calculated_host_listings_count_entire_homes  \
0                                           2   
1                                         643   
2                                          51   
3                                          14   
4                                          19   

  calculated_host_listings_count_private_rooms  \
0                                           42   
1                                            0   
2                                            0   
3                                            0   
4                                            0   

   calculated_host_listings_count_shared_rooms reviews_per_month  
0                                            0               NaN  
1                                            0               NaN  
2                                            0              1.61  
3                                            0              0.19  
4                                            0              0.16  

[5 rows x 52 columns]

In [8]:
pd.set_option('display.max_columns', None)

# train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)

train['host_response_rate'] = train['host_response_rate'].str.rstrip('%').astype(float)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float)
train['host_since'] = pd.to_datetime(train['host_since'])

train['host_is_superhost'] = train['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

train['instant_bookable'] = train['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
train['host_has_profile_pic'] = train['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
train['host_identity_verified'] = train['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
train['has_availability'] = train['has_availability'].apply(lambda x: 1 if x == 't' else 0)


'------------------------------------------------------------------------------------------------'

test['host_response_rate'] = test['host_response_rate'].str.rstrip('%').astype(float)
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.rstrip('%').astype(float)
test['host_since'] = pd.to_datetime(test['host_since'])

test['instant_bookable'] = test['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
test['host_has_profile_pic'] = test['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
test['host_identity_verified'] = test['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
test['has_availability'] = test['has_availability'].apply(lambda x: 1 if x == 't' else 0)


# test['host_is_superhost'] = test['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)


# covert host_since into a usuable form
current_date = pd.to_datetime("today")

train['days_since_host_started'] = (current_date - train['host_since']).dt.days
test['days_since_host_started'] = (current_date - test['host_since']).dt.days

# Impute missing values

#Impute numerical columns with the mean
for column in train.select_dtypes(include=['number']).columns:
    train[column].fillna(train[column].mean(), inplace=True)
    
for column in test.select_dtypes(include=['number']).columns:
    test[column].fillna(train[column].mean(), inplace=True)  # Use train's mean to avoid data leakage

# Impute categorical columns with the mode (most frequent value)
for column in train.select_dtypes(include=['object', 'category']).columns:
    train[column].fillna(train[column].mode()[0], inplace=True)
    
for column in test.select_dtypes(include=['object', 'category']).columns:
    test[column].fillna(train[column].mode()[0], inplace=True)  # Use train's mode to avoid data leakage


# print(test.isnull().any())
# print('---------------------------------------------------------')
# print(train.isnull().any())

def extract_bathrooms(text):
    if 'half' in text.lower():
        return 0.5
    else:
        match = re.search(r'\d+\.?\d*', text)
        return float(match.group()) if match else np.nan

train['bathrooms'] = train['bathrooms_text'].apply(extract_bathrooms)
test['bathrooms'] = test['bathrooms_text'].apply(extract_bathrooms)

train['bathrooms'].fillna(train['bathrooms'].mean(), inplace=True)
test['bathrooms'].fillna(test['bathrooms'].mean(), inplace=True)


train_categories = set(train['host_location'].unique())
test['host_location'] = test['host_location'].apply(lambda x: x if x in train_categories else np.nan)
test['host_location'].fillna(train['host_location'].mode()[0], inplace=True)

train['host_verifications_count'] = train['host_verifications'].apply(lambda x: len(x.split(',')))
test['host_verifications_count'] = test['host_verifications'].apply(lambda x: len(x.split(',')))

train.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)
test.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)

train = pd.get_dummies(train)
test = pd.get_dummies(test)




train.head()

id  host_response_rate  host_acceptance_rate  \
0            107745711                96.0                  95.0   
1             78539203               100.0                  97.0   
2  1112961466060884803               100.0                 100.0   
3            104794437               100.0                  98.0   
4  1491898404543717797               100.0                 100.0   

   host_is_superhost  host_listings_count  host_total_listings_count  \
0                  0                   11                         13   
1                  0                 1141                       2346   
2                  0                    1                          1   
3                  0                   55                         58   
4                  1                   74                         75   

   host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                     1                       1  41.794240 -87.661380   
1                     1                       1  41.892749 -87.620711   
2                     1                       1  41.948160 -87.652309   
3                     1                       1  41.801030 -87.597160   
4                     1                       1  41.855770 -87.624130   

   accommodates  beds  minimum_nights  maximum_nights  minimum_minimum_nights  \
0             1   1.0              32            1125                      32   
1            12   3.0              32             365                      32   
2             6   3.0               2              45                       2   
3             2   1.0               2             180                       2   
4             6   3.0               2             365                       2   

   maximum_minimum_nights  minimum_maximum_nights  maximum_maximum_nights  \
0                      32                    1125                    1125   
1                      32                     365                     365   
2                       2                      45                      45   
3                       2                     180                     180   
4                       2                    1125                    1125   

   minimum_nights_avg_ntm  maximum_nights_avg_ntm  has_availability  \
0                    32.0                  1125.0                 1   
1                    32.0                   365.0                 1   
2                     2.0                    45.0                 1   
3                     2.0                   180.0                 1   
4                     2.0                  1125.0                 1   

   availability_30  availability_60  availability_90  availability_365  \
0               30               60               90               365   
1               25               49               66               135   
2                0                0                0                 0   
3               23               53               83               355   
4                0                0                0                 0   

   number_of_reviews  number_of_reviews_ltm  number_of_reviews_l30d  \
0                 18                      0                       0   
1                  0                      0                       0   
2                 14                     14                       1   
3                 13                     13                       3   
4                 64                     42                       3   

   review_scores_rating  review_scores_accuracy  review_scores_cleanliness  \
0              4.940000                5.000000                   4.890000   
1              4.760427                4.794096                   4.748194   
2              4.870000                5.000000                   5.000000   
3              4.080000                4.080000                   4.000000   
4              4.800000                4.740000                   4.750000   

   review_scores_chec

In [9]:
train_cols = list(train.columns)
test_cols = list(test.columns)

train_cols = [col for col in train_cols if col in test_cols]
test_cols = [col for col in test_cols if col in train_cols]

train_cols == test_cols

True

In [10]:
x_train = train[train_cols].drop(columns=['id'])
y_train = train.host_is_superhost

x_test = test[test_cols].drop(columns=['id'])

# Remove restricted characters
x_train.columns = x_train.columns.str.replace('[', '', regex=True).str.replace(']', '', regex=True).str.replace('<', '', regex=True)
x_test.columns = x_test.columns.str.replace('[', '', regex=True).str.replace(']', '', regex=True).str.replace('<', '', regex=True)

In [11]:
pos = y_train.value_counts()[0] / y_train.value_counts()[1]

### Predictor Selection

In [12]:
model1 = CatBoostClassifier(
    
    scale_pos_weight = pos, 
    random_state=1,
    n_estimators=20000,
    max_depth=6,
    subsample=0.75,
    learning_rate=0.1,
    reg_lambda=0.1
                         )

model1.fit(x_train, y_train)

0:	learn: 0.6261657	total: 138ms	remaining: 45m 58s
1:	learn: 0.5878344	total: 142ms	remaining: 23m 41s
2:	learn: 0.5582030	total: 146ms	remaining: 16m 10s
3:	learn: 0.5358778	total: 149ms	remaining: 12m 26s
4:	learn: 0.5142405	total: 153ms	remaining: 10m 11s
5:	learn: 0.4991734	total: 156ms	remaining: 8m 41s
6:	learn: 0.4836904	total: 160ms	remaining: 7m 37s
7:	learn: 0.4670998	total: 164ms	remaining: 6m 49s
8:	learn: 0.4486722	total: 167ms	remaining: 6m 11s
9:	learn: 0.4385933	total: 171ms	remaining: 5m 41s
10:	learn: 0.4283557	total: 174ms	remaining: 5m 16s
11:	learn: 0.4204217	total: 178ms	remaining: 4m 55s
12:	learn: 0.4109728	total: 181ms	remaining: 4m 38s
13:	learn: 0.4019571	total: 185ms	remaining: 4m 23s
14:	learn: 0.3935659	total: 188ms	remaining: 4m 11s
15:	learn: 0.3852017	total: 192ms	remaining: 3m 59s
16:	learn: 0.3792633	total: 195ms	remaining: 3m 49s
17:	learn: 0.3743928	total: 199ms	remaining: 3m 40s
18:	learn: 0.3679872	total: 202ms	remaining: 3m 32s
19:	learn: 0.3652

169:	learn: 0.1384128	total: 757ms	remaining: 1m 28s
170:	learn: 0.1371565	total: 761ms	remaining: 1m 28s
171:	learn: 0.1362688	total: 764ms	remaining: 1m 28s
172:	learn: 0.1355246	total: 769ms	remaining: 1m 28s
173:	learn: 0.1350193	total: 772ms	remaining: 1m 28s
174:	learn: 0.1347463	total: 776ms	remaining: 1m 27s
175:	learn: 0.1340911	total: 780ms	remaining: 1m 27s
176:	learn: 0.1335821	total: 784ms	remaining: 1m 27s
177:	learn: 0.1327504	total: 788ms	remaining: 1m 27s
178:	learn: 0.1322537	total: 791ms	remaining: 1m 27s
179:	learn: 0.1317404	total: 795ms	remaining: 1m 27s
180:	learn: 0.1308233	total: 798ms	remaining: 1m 27s
181:	learn: 0.1305087	total: 802ms	remaining: 1m 27s
182:	learn: 0.1294937	total: 806ms	remaining: 1m 27s
183:	learn: 0.1292393	total: 809ms	remaining: 1m 27s
184:	learn: 0.1282279	total: 813ms	remaining: 1m 27s
185:	learn: 0.1278743	total: 816ms	remaining: 1m 26s
186:	learn: 0.1272667	total: 819ms	remaining: 1m 26s
187:	learn: 0.1265850	total: 822ms	remaining: 

338:	learn: 0.0615785	total: 1.36s	remaining: 1m 19s
339:	learn: 0.0613923	total: 1.37s	remaining: 1m 19s
340:	learn: 0.0610081	total: 1.37s	remaining: 1m 19s
341:	learn: 0.0608767	total: 1.38s	remaining: 1m 19s
342:	learn: 0.0606815	total: 1.38s	remaining: 1m 19s
343:	learn: 0.0605252	total: 1.38s	remaining: 1m 19s
344:	learn: 0.0600805	total: 1.39s	remaining: 1m 19s
345:	learn: 0.0598178	total: 1.39s	remaining: 1m 18s
346:	learn: 0.0596730	total: 1.39s	remaining: 1m 18s
347:	learn: 0.0595411	total: 1.4s	remaining: 1m 18s
348:	learn: 0.0591994	total: 1.4s	remaining: 1m 18s
349:	learn: 0.0590598	total: 1.4s	remaining: 1m 18s
350:	learn: 0.0589065	total: 1.41s	remaining: 1m 18s
351:	learn: 0.0585177	total: 1.41s	remaining: 1m 18s
352:	learn: 0.0582181	total: 1.41s	remaining: 1m 18s
353:	learn: 0.0579789	total: 1.42s	remaining: 1m 18s
354:	learn: 0.0577009	total: 1.42s	remaining: 1m 18s
355:	learn: 0.0573870	total: 1.43s	remaining: 1m 18s
356:	learn: 0.0571637	total: 1.43s	remaining: 1m 

507:	learn: 0.0294598	total: 1.99s	remaining: 1m 16s
508:	learn: 0.0292684	total: 1.99s	remaining: 1m 16s
509:	learn: 0.0290836	total: 1.99s	remaining: 1m 16s
510:	learn: 0.0289610	total: 2s	remaining: 1m 16s
511:	learn: 0.0288643	total: 2s	remaining: 1m 16s
512:	learn: 0.0286976	total: 2s	remaining: 1m 16s
513:	learn: 0.0286410	total: 2.01s	remaining: 1m 16s
514:	learn: 0.0285971	total: 2.01s	remaining: 1m 16s
515:	learn: 0.0284358	total: 2.01s	remaining: 1m 16s
516:	learn: 0.0283570	total: 2.02s	remaining: 1m 16s
517:	learn: 0.0282728	total: 2.02s	remaining: 1m 16s
518:	learn: 0.0281920	total: 2.02s	remaining: 1m 16s
519:	learn: 0.0281528	total: 2.03s	remaining: 1m 15s
520:	learn: 0.0280654	total: 2.03s	remaining: 1m 15s
521:	learn: 0.0278646	total: 2.03s	remaining: 1m 15s
522:	learn: 0.0277908	total: 2.04s	remaining: 1m 15s
523:	learn: 0.0277275	total: 2.04s	remaining: 1m 15s
524:	learn: 0.0276869	total: 2.04s	remaining: 1m 15s
525:	learn: 0.0276476	total: 2.05s	remaining: 1m 15s
52

674:	learn: 0.0161702	total: 2.59s	remaining: 1m 14s
675:	learn: 0.0161009	total: 2.6s	remaining: 1m 14s
676:	learn: 0.0160620	total: 2.6s	remaining: 1m 14s
677:	learn: 0.0159638	total: 2.6s	remaining: 1m 14s
678:	learn: 0.0159097	total: 2.61s	remaining: 1m 14s
679:	learn: 0.0158677	total: 2.61s	remaining: 1m 14s
680:	learn: 0.0158140	total: 2.62s	remaining: 1m 14s
681:	learn: 0.0157416	total: 2.62s	remaining: 1m 14s
682:	learn: 0.0156722	total: 2.62s	remaining: 1m 14s
683:	learn: 0.0156092	total: 2.63s	remaining: 1m 14s
684:	learn: 0.0155637	total: 2.63s	remaining: 1m 14s
685:	learn: 0.0154967	total: 2.63s	remaining: 1m 14s
686:	learn: 0.0154720	total: 2.64s	remaining: 1m 14s
687:	learn: 0.0154546	total: 2.64s	remaining: 1m 14s
688:	learn: 0.0154191	total: 2.65s	remaining: 1m 14s
689:	learn: 0.0153621	total: 2.65s	remaining: 1m 14s
690:	learn: 0.0153115	total: 2.65s	remaining: 1m 14s
691:	learn: 0.0152912	total: 2.66s	remaining: 1m 14s
692:	learn: 0.0152353	total: 2.66s	remaining: 1m 

...

19017:	learn: 0.0020484	total: 1m 1s	remaining: 3.17s
19018:	learn: 0.0020484	total: 1m 1s	remaining: 3.17s
19019:	learn: 0.0020484	total: 1m 1s	remaining: 3.17s
19020:	learn: 0.0020484	total: 1m 1s	remaining: 3.16s
19021:	learn: 0.0020484	total: 1m 1s	remaining: 3.16s
19022:	learn: 0.0020484	total: 1m 1s	remaining: 3.15s
19023:	learn: 0.0020484	total: 1m 1s	remaining: 3.15s
19024:	learn: 0.0020484	total: 1m 1s	remaining: 3.15s
19025:	learn: 0.0020484	total: 1m 1s	remaining: 3.15s
19026:	learn: 0.0020484	total: 1m 1s	remaining: 3.14s
19027:	learn: 0.0020484	total: 1m 1s	remaining: 3.14s
19028:	learn: 0.0020484	total: 1m 1s	remaining: 3.13s
19029:	learn: 0.0020483	total: 1m 1s	remaining: 3.13s
19030:	learn: 0.0020483	total: 1m 1s	remaining: 3.13s
19031:	learn: 0.0020483	total: 1m 1s	remaining: 3.13s
19032:	learn: 0.0020483	total: 1m 1s	remaining: 3.12s
19033:	learn: 0.0020483	total: 1m 1s	remaining: 3.12s
19034:	learn: 0.0020483	total: 1m 1s	remaining: 3.12s
19035:	learn: 0.0020483	tota

19213:	learn: 0.0020391	total: 1m 2s	remaining: 2.54s
19214:	learn: 0.0020391	total: 1m 2s	remaining: 2.53s
19215:	learn: 0.0020391	total: 1m 2s	remaining: 2.53s
19216:	learn: 0.0020391	total: 1m 2s	remaining: 2.53s
19217:	learn: 0.0020391	total: 1m 2s	remaining: 2.52s
19218:	learn: 0.0020391	total: 1m 2s	remaining: 2.52s
19219:	learn: 0.0020391	total: 1m 2s	remaining: 2.52s
19220:	learn: 0.0020391	total: 1m 2s	remaining: 2.51s
19221:	learn: 0.0020391	total: 1m 2s	remaining: 2.51s
19222:	learn: 0.0020390	total: 1m 2s	remaining: 2.51s
19223:	learn: 0.0020389	total: 1m 2s	remaining: 2.5s
19224:	learn: 0.0020389	total: 1m 2s	remaining: 2.5s
19225:	learn: 0.0020389	total: 1m 2s	remaining: 2.5s
19226:	learn: 0.0020389	total: 1m 2s	remaining: 2.5s
19227:	learn: 0.0020389	total: 1m 2s	remaining: 2.49s
19228:	learn: 0.0020389	total: 1m 2s	remaining: 2.49s
19229:	learn: 0.0020389	total: 1m 2s	remaining: 2.48s
19230:	learn: 0.0020389	total: 1m 2s	remaining: 2.48s
19231:	learn: 0.0020389	total: 1

19396:	learn: 0.0020379	total: 1m 2s	remaining: 1.95s
19397:	learn: 0.0020379	total: 1m 2s	remaining: 1.94s
19398:	learn: 0.0020379	total: 1m 2s	remaining: 1.94s
19399:	learn: 0.0020379	total: 1m 2s	remaining: 1.94s
19400:	learn: 0.0020379	total: 1m 2s	remaining: 1.93s
19401:	learn: 0.0020379	total: 1m 2s	remaining: 1.93s
19402:	learn: 0.0020379	total: 1m 2s	remaining: 1.93s
19403:	learn: 0.0020379	total: 1m 2s	remaining: 1.92s
19404:	learn: 0.0020379	total: 1m 2s	remaining: 1.92s
19405:	learn: 0.0020379	total: 1m 2s	remaining: 1.92s
19406:	learn: 0.0020379	total: 1m 2s	remaining: 1.91s
19407:	learn: 0.0020379	total: 1m 2s	remaining: 1.91s
19408:	learn: 0.0020378	total: 1m 2s	remaining: 1.91s
19409:	learn: 0.0020378	total: 1m 2s	remaining: 1.9s
19410:	learn: 0.0020378	total: 1m 2s	remaining: 1.9s
19411:	learn: 0.0020378	total: 1m 2s	remaining: 1.9s
19412:	learn: 0.0020378	total: 1m 2s	remaining: 1.9s
19413:	learn: 0.0020378	total: 1m 2s	remaining: 1.89s
19414:	learn: 0.0020378	total: 1

19597:	learn: 0.0020367	total: 1m 3s	remaining: 1.3s
19598:	learn: 0.0020367	total: 1m 3s	remaining: 1.29s
19599:	learn: 0.0020367	total: 1m 3s	remaining: 1.29s
19600:	learn: 0.0020366	total: 1m 3s	remaining: 1.29s
19601:	learn: 0.0020366	total: 1m 3s	remaining: 1.28s
19602:	learn: 0.0020366	total: 1m 3s	remaining: 1.28s
19603:	learn: 0.0020366	total: 1m 3s	remaining: 1.28s
19604:	learn: 0.0020366	total: 1m 3s	remaining: 1.27s
19605:	learn: 0.0020366	total: 1m 3s	remaining: 1.27s
19606:	learn: 0.0020366	total: 1m 3s	remaining: 1.27s
19607:	learn: 0.0020366	total: 1m 3s	remaining: 1.26s
19608:	learn: 0.0020366	total: 1m 3s	remaining: 1.26s
19609:	learn: 0.0020366	total: 1m 3s	remaining: 1.26s
19610:	learn: 0.0020366	total: 1m 3s	remaining: 1.25s
19611:	learn: 0.0020366	total: 1m 3s	remaining: 1.25s
19612:	learn: 0.0020366	total: 1m 3s	remaining: 1.25s
19613:	learn: 0.0020366	total: 1m 3s	remaining: 1.25s
19614:	learn: 0.0020365	total: 1m 3s	remaining: 1.24s
19615:	learn: 0.0020365	total

19797:	learn: 0.0020304	total: 1m 3s	remaining: 651ms
19798:	learn: 0.0020304	total: 1m 3s	remaining: 648ms
19799:	learn: 0.0020305	total: 1m 3s	remaining: 645ms
19800:	learn: 0.0020304	total: 1m 3s	remaining: 642ms
19801:	learn: 0.0020304	total: 1m 3s	remaining: 638ms
19802:	learn: 0.0020304	total: 1m 3s	remaining: 635ms
19803:	learn: 0.0020304	total: 1m 3s	remaining: 632ms
19804:	learn: 0.0020304	total: 1m 3s	remaining: 629ms
19805:	learn: 0.0020304	total: 1m 3s	remaining: 626ms
19806:	learn: 0.0020304	total: 1m 3s	remaining: 622ms
19807:	learn: 0.0020304	total: 1m 3s	remaining: 619ms
19808:	learn: 0.0020304	total: 1m 3s	remaining: 616ms
19809:	learn: 0.0020304	total: 1m 3s	remaining: 613ms
19810:	learn: 0.0020304	total: 1m 3s	remaining: 609ms
19811:	learn: 0.0020304	total: 1m 3s	remaining: 606ms
19812:	learn: 0.0020304	total: 1m 3s	remaining: 603ms
19813:	learn: 0.0020304	total: 1m 3s	remaining: 600ms
19814:	learn: 0.0020303	total: 1m 3s	remaining: 597ms
19815:	learn: 0.0020303	tota

In [13]:
feature_importances = model1.feature_importances_

features_importance = pd.DataFrame({'feature': x_train.columns, 'importance': feature_importances})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

features_importance

feature  importance
2                      host_listings_count    8.032055
1                     host_acceptance_rate    6.976301
39                 days_since_host_started    6.658844
3                host_total_listings_count    5.565336
24                   number_of_reviews_ltm    4.920442
..                                     ...         ...
294       neighbourhood_cleansed_Hegewisch    0.000000
149           host_neighbourhood_East Side    0.000000
156           host_neighbourhood_Gage Park    0.000000
291  neighbourhood_cleansed_Garfield Ridge    0.000000
380                  room_type_Shared room    0.000000

[381 rows x 2 columns]

In [14]:
important_features = list(features_importance.head(20).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

### Model Tuning and Training

In [16]:
model = CatBoostClassifier(random_state=1, scale_pos_weight=pos)

grid = {
    'n_estimators' : [500, 1000, 10000],
    'max_depth' : [4, 6],
    'learning_rate' : [0.05, 0.1, 1],
    'subsample' : [0.75, 1.0],
    'reg_lambda':[0, 0.1, 1], 
}

gscv = GridSearchCV(model, grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
0:	learn: 0.6653124	total: 4.35ms	remaining: 43.5s
1:	learn: 0.6359694	total: 7.93ms	remaining: 39.6s
2:	learn: 0.6113783	total: 11.7ms	remaining: 38.9s
3:	learn: 0.5916198	total: 15ms	remaining: 37.5s
4:	learn: 0.5719888	total: 18.5ms	remaining: 36.9s
5:	learn: 0.5595595	total: 22ms	remaining: 36.7s
6:	learn: 0.5420784	total: 25.8ms	remaining: 36.8s
7:	learn: 0.5278988	total: 29.4ms	remaining: 36.7s
8:	learn: 0.5148606	total: 32.9ms	remaining: 36.5s
9:	learn: 0.5046866	total: 36.7ms	remaining: 36.6s
10:	learn: 0.4960072	total: 40.2ms	remaining: 36.5s
11:	learn: 0.4870269	total: 43.8ms	remaining: 36.4s
12:	learn: 0.4784050	total: 47.4ms	remaining: 36.4s
13:	learn: 0.4714517	total: 50.3ms	remaining: 35.9s
14:	learn: 0.4645933	total: 53.9ms	remaining: 35.9s
15:	learn: 0.4595692	total: 57.4ms	remaining: 35.8s
16:	learn: 0.4521409	total: 61.3ms	remaining: 36s
17:	learn: 0.4473779	total: 65ms	remaining: 36.1s
18:	learn: 0.441770

168:	learn: 0.2214984	total: 601ms	remaining: 35s
169:	learn: 0.2211525	total: 605ms	remaining: 35s
170:	learn: 0.2203346	total: 608ms	remaining: 34.9s
171:	learn: 0.2197412	total: 612ms	remaining: 34.9s
172:	learn: 0.2192910	total: 615ms	remaining: 34.9s
173:	learn: 0.2186721	total: 619ms	remaining: 34.9s
174:	learn: 0.2180793	total: 622ms	remaining: 34.9s
175:	learn: 0.2168933	total: 625ms	remaining: 34.9s
176:	learn: 0.2163863	total: 628ms	remaining: 34.9s
177:	learn: 0.2156395	total: 632ms	remaining: 34.9s
178:	learn: 0.2151620	total: 635ms	remaining: 34.8s
179:	learn: 0.2143225	total: 639ms	remaining: 34.8s
180:	learn: 0.2134024	total: 642ms	remaining: 34.8s
181:	learn: 0.2125003	total: 646ms	remaining: 34.9s
182:	learn: 0.2115494	total: 650ms	remaining: 34.9s
183:	learn: 0.2111259	total: 654ms	remaining: 34.9s
184:	learn: 0.2107103	total: 657ms	remaining: 34.9s
185:	learn: 0.2100080	total: 661ms	remaining: 34.9s
186:	learn: 0.2094373	total: 664ms	remaining: 34.8s
187:	learn: 0.20

337:	learn: 0.1411151	total: 1.21s	remaining: 34.6s
338:	learn: 0.1408199	total: 1.21s	remaining: 34.6s
339:	learn: 0.1405644	total: 1.22s	remaining: 34.6s
340:	learn: 0.1403659	total: 1.22s	remaining: 34.6s
341:	learn: 0.1400885	total: 1.23s	remaining: 34.6s
342:	learn: 0.1395751	total: 1.23s	remaining: 34.6s
343:	learn: 0.1392973	total: 1.23s	remaining: 34.6s
344:	learn: 0.1389741	total: 1.24s	remaining: 34.6s
345:	learn: 0.1385110	total: 1.24s	remaining: 34.6s
346:	learn: 0.1382444	total: 1.24s	remaining: 34.6s
347:	learn: 0.1381114	total: 1.25s	remaining: 34.6s
348:	learn: 0.1379161	total: 1.25s	remaining: 34.6s
349:	learn: 0.1375884	total: 1.25s	remaining: 34.6s
350:	learn: 0.1372444	total: 1.26s	remaining: 34.6s
351:	learn: 0.1368757	total: 1.26s	remaining: 34.6s
352:	learn: 0.1364674	total: 1.27s	remaining: 34.6s
353:	learn: 0.1362156	total: 1.27s	remaining: 34.6s
354:	learn: 0.1358726	total: 1.27s	remaining: 34.6s
355:	learn: 0.1355963	total: 1.28s	remaining: 34.6s
356:	learn: 

546:	learn: 0.0895007	total: 1.99s	remaining: 34.3s
547:	learn: 0.0893947	total: 1.99s	remaining: 34.3s
548:	learn: 0.0892281	total: 1.99s	remaining: 34.3s
549:	learn: 0.0890581	total: 2s	remaining: 34.3s
550:	learn: 0.0888894	total: 2s	remaining: 34.3s
551:	learn: 0.0886918	total: 2s	remaining: 34.3s
552:	learn: 0.0886149	total: 2.01s	remaining: 34.3s
553:	learn: 0.0884499	total: 2.01s	remaining: 34.3s
554:	learn: 0.0882715	total: 2.02s	remaining: 34.3s
555:	learn: 0.0880892	total: 2.02s	remaining: 34.3s
556:	learn: 0.0879039	total: 2.02s	remaining: 34.3s
557:	learn: 0.0877035	total: 2.02s	remaining: 34.3s
558:	learn: 0.0875635	total: 2.03s	remaining: 34.3s
559:	learn: 0.0874155	total: 2.03s	remaining: 34.3s
560:	learn: 0.0872472	total: 2.04s	remaining: 34.3s
561:	learn: 0.0870834	total: 2.04s	remaining: 34.3s
562:	learn: 0.0869103	total: 2.04s	remaining: 34.3s
563:	learn: 0.0867927	total: 2.05s	remaining: 34.2s
564:	learn: 0.0866620	total: 2.05s	remaining: 34.2s
565:	learn: 0.0864707

710:	learn: 0.0639278	total: 2.6s	remaining: 34s
711:	learn: 0.0637652	total: 2.6s	remaining: 34s
712:	learn: 0.0636753	total: 2.61s	remaining: 34s
713:	learn: 0.0635165	total: 2.61s	remaining: 34s
714:	learn: 0.0633426	total: 2.62s	remaining: 34s
715:	learn: 0.0632012	total: 2.62s	remaining: 34s
716:	learn: 0.0630839	total: 2.62s	remaining: 34s
717:	learn: 0.0629871	total: 2.63s	remaining: 34s
718:	learn: 0.0629132	total: 2.63s	remaining: 33.9s
719:	learn: 0.0628372	total: 2.63s	remaining: 33.9s
720:	learn: 0.0627223	total: 2.64s	remaining: 33.9s
721:	learn: 0.0626370	total: 2.64s	remaining: 33.9s
722:	learn: 0.0625756	total: 2.64s	remaining: 33.9s
723:	learn: 0.0624523	total: 2.65s	remaining: 33.9s
724:	learn: 0.0623575	total: 2.65s	remaining: 33.9s
725:	learn: 0.0622253	total: 2.65s	remaining: 33.9s
726:	learn: 0.0621175	total: 2.66s	remaining: 33.9s
727:	learn: 0.0620615	total: 2.66s	remaining: 33.9s
728:	learn: 0.0619684	total: 2.67s	remaining: 33.9s
729:	learn: 0.0617602	total: 2

...

9277:	learn: 0.0042901	total: 26.5s	remaining: 2.06s
9278:	learn: 0.0042900	total: 26.5s	remaining: 2.06s
9279:	learn: 0.0042900	total: 26.5s	remaining: 2.06s
9280:	learn: 0.0042900	total: 26.5s	remaining: 2.05s
9281:	learn: 0.0042900	total: 26.5s	remaining: 2.05s
9282:	learn: 0.0042900	total: 26.5s	remaining: 2.05s
9283:	learn: 0.0042900	total: 26.5s	remaining: 2.05s
9284:	learn: 0.0042900	total: 26.5s	remaining: 2.04s
9285:	learn: 0.0042900	total: 26.5s	remaining: 2.04s
9286:	learn: 0.0042852	total: 26.6s	remaining: 2.04s
9287:	learn: 0.0042852	total: 26.6s	remaining: 2.04s
9288:	learn: 0.0042852	total: 26.6s	remaining: 2.03s
9289:	learn: 0.0042852	total: 26.6s	remaining: 2.03s
9290:	learn: 0.0042852	total: 26.6s	remaining: 2.03s
9291:	learn: 0.0042852	total: 26.6s	remaining: 2.02s
9292:	learn: 0.0042852	total: 26.6s	remaining: 2.02s
9293:	learn: 0.0042852	total: 26.6s	remaining: 2.02s
9294:	learn: 0.0042852	total: 26.6s	remaining: 2.02s
9295:	learn: 0.0042852	total: 26.6s	remaining:

9495:	learn: 0.0042753	total: 27.1s	remaining: 1.44s
9496:	learn: 0.0042753	total: 27.1s	remaining: 1.44s
9497:	learn: 0.0042753	total: 27.1s	remaining: 1.43s
9498:	learn: 0.0042752	total: 27.1s	remaining: 1.43s
9499:	learn: 0.0042752	total: 27.1s	remaining: 1.43s
9500:	learn: 0.0042752	total: 27.1s	remaining: 1.42s
9501:	learn: 0.0042752	total: 27.1s	remaining: 1.42s
9502:	learn: 0.0042752	total: 27.1s	remaining: 1.42s
9503:	learn: 0.0042752	total: 27.1s	remaining: 1.42s
9504:	learn: 0.0042752	total: 27.1s	remaining: 1.41s
9505:	learn: 0.0042752	total: 27.1s	remaining: 1.41s
9506:	learn: 0.0042752	total: 27.1s	remaining: 1.41s
9507:	learn: 0.0042752	total: 27.1s	remaining: 1.4s
9508:	learn: 0.0042752	total: 27.1s	remaining: 1.4s
9509:	learn: 0.0042752	total: 27.1s	remaining: 1.4s
9510:	learn: 0.0042752	total: 27.1s	remaining: 1.4s
9511:	learn: 0.0042752	total: 27.1s	remaining: 1.39s
9512:	learn: 0.0042752	total: 27.1s	remaining: 1.39s
9513:	learn: 0.0042752	total: 27.1s	remaining: 1.3

9655:	learn: 0.0042740	total: 27.5s	remaining: 979ms
9656:	learn: 0.0042740	total: 27.5s	remaining: 977ms
9657:	learn: 0.0042740	total: 27.5s	remaining: 974ms
9658:	learn: 0.0042740	total: 27.5s	remaining: 971ms
9659:	learn: 0.0042739	total: 27.5s	remaining: 968ms
9660:	learn: 0.0042739	total: 27.5s	remaining: 965ms
9661:	learn: 0.0042739	total: 27.5s	remaining: 962ms
9662:	learn: 0.0042739	total: 27.5s	remaining: 959ms
9663:	learn: 0.0042739	total: 27.5s	remaining: 957ms
9664:	learn: 0.0042739	total: 27.5s	remaining: 954ms
9665:	learn: 0.0042739	total: 27.5s	remaining: 951ms
9666:	learn: 0.0042739	total: 27.5s	remaining: 948ms
9667:	learn: 0.0042739	total: 27.5s	remaining: 945ms
9668:	learn: 0.0042739	total: 27.5s	remaining: 942ms
9669:	learn: 0.0042739	total: 27.5s	remaining: 939ms
9670:	learn: 0.0042739	total: 27.5s	remaining: 937ms
9671:	learn: 0.0042739	total: 27.5s	remaining: 934ms
9672:	learn: 0.0042739	total: 27.5s	remaining: 931ms
9673:	learn: 0.0042739	total: 27.5s	remaining:

9818:	learn: 0.0042725	total: 27.9s	remaining: 514ms
9819:	learn: 0.0042725	total: 27.9s	remaining: 511ms
9820:	learn: 0.0042725	total: 27.9s	remaining: 508ms
9821:	learn: 0.0042725	total: 27.9s	remaining: 506ms
9822:	learn: 0.0042725	total: 27.9s	remaining: 503ms
9823:	learn: 0.0042724	total: 27.9s	remaining: 500ms
9824:	learn: 0.0042724	total: 27.9s	remaining: 497ms
9825:	learn: 0.0042724	total: 27.9s	remaining: 494ms
9826:	learn: 0.0042724	total: 27.9s	remaining: 491ms
9827:	learn: 0.0042724	total: 27.9s	remaining: 489ms
9828:	learn: 0.0042725	total: 27.9s	remaining: 486ms
9829:	learn: 0.0042724	total: 27.9s	remaining: 483ms
9830:	learn: 0.0042724	total: 27.9s	remaining: 480ms
9831:	learn: 0.0042724	total: 27.9s	remaining: 477ms
9832:	learn: 0.0042724	total: 27.9s	remaining: 474ms
9833:	learn: 0.0042724	total: 27.9s	remaining: 471ms
9834:	learn: 0.0042724	total: 27.9s	remaining: 469ms
9835:	learn: 0.0042724	total: 27.9s	remaining: 466ms
9836:	learn: 0.0042724	total: 27.9s	remaining:

{'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 10000, 'reg_lambda': 1, 'subsample': 0.75}
0.9003415712276471


### Prediction

In [17]:
ids = test.id
results = pd.DataFrame({
    'id': ids,
    'predicted': y_pred
})


results.to_csv('Catboost_Classifier_submission.csv', index=False)
results.head(30)

id  predicted
0   1543972437713169913          0
1   1710552057351883447          0
2              97075525          1
3              83734823          1
4              56722823          0
5             103823601          1
6   1159028243437550345          0
7              92485765          0
8              11142401          1
9   1189875082289533291          0
10  1544820103115167001          1
11             26331127          0
12             58383089          1
13             64762469          0
14            100902043          1
15            103130147          0
16             94352581          1
17  1849015146225985937          0
18             91307779          1
19             53591385          0
20             84071033          0
21             99484949          0
22             97864023          0
23             31809543          0
24  1519081708190436577          1
25             59771473          1
26  1310888620900225001          0
27             23054201          1
28             37470529          0
29  1212015341587401973          1